#### 一. Enabling Preemption
* YARN FairScheduler会抢占在某些队列中处于running状态的container, 如果以下三个条件均满足:
    * 该队列申请到的资源超过其Instantaneous FairShare
    * 有1个活多个处于饥饿状态的app
    * 开启preemption
* 集群利用率通常是开启preemption的先决条件, 在利用率低的集群上开启container抢占机制会引起不必要的container波动, 集群利用率通常是由vCores 或memory中使用较大的那个值

* 队列可以声明成不能被强占,来突出他的高优先级

* 有两种类型的饥饿 starvation
    * FairShare Starvation
    * MinShare Starvation

#### 二. 两种饥饿
* FairShare Starvation
    1. 一个app是FairShare Starvation的, 如果以下条件均满足:
        * app所需资源并未得到满足
        * app的占用资源小于他的Instantaneous FairShare(整数)乘以FairShare Preemption Threshold(百分比)  
        * app满足以上两个条件至少持续了FairShare Preemption Timeout时间   
          FairShare Preemption Timeout默认不去配置, FairShare Preemption Threshold默认为0.5,   
          FairShare Preemption Timeout为了延缓资源抢占的时间
  
    假设已配置2个参数:  FairShare Preemption Timeout=5, FairShare Preemption Threshold=0.8, 这表示如果app在5秒内还没占用到其Instantaneous FairShare的80%的资源, 

    2. 开启preemption, 并不能保证队列中的app获得其全部的Instantaneous FairShare, 仅仅表示在preemption timeout超时时间后, app能占用其`Instantaneous FairShare*阈值` 的资源

* MinShare Starvation
    1. 一个队列是MinShare Starvation的, 如果同时满足以下条件
        * 队列中至少有一个app存在不满足的资源要求的
        * 队列的资源占用小于其MinShare
        * 队列的资源占用小于其MinShare持续了至少MinShare Preemption Timeout的时间
    2. MinShare Preemption Timeout设置为了延缓进行资源抢占的时间


* 一个container可以被强占, 除非以下2个条件同时满足
    * 这个container的作业所在的queue是可以被强占的
    * 杀死container不会使得app占用的资源小于其fair share(即不会因为被抢占而变成fairshare starv)
    * 无论是否配置了fair share饥饿和minshare饥饿, 都只有占用资源大于其Instantaneous FairShar出的作业可以被强占